In [1]:
# Import Libraries
import numpy as np
import json

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


In [2]:
#Upload VGG16

vgg16 = VGG16(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
vgg16.save('vgg16.h5')

In [4]:
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [9]:
y = prepare_image('C:/Users/Ashar/Desktop/image1.jpg')

In [10]:
preds = vgg16.predict(y)

In [12]:
preds

array([[1.36774520e-06, 5.05270327e-06, 3.98542113e-07, 2.91700900e-07,
        4.99147347e-07, 5.10505970e-07, 4.26944979e-07, 5.15141837e-05,
        2.53420876e-05, 1.01560763e-05, 1.57050283e-06, 1.42483805e-06,
        1.48749441e-05, 2.80487939e-06, 5.24418328e-06, 1.41202572e-05,
        4.81278767e-06, 6.69026622e-06, 1.33623289e-05, 4.34456342e-06,
        4.17613046e-06, 4.66127631e-06, 1.16498859e-05, 1.08100612e-04,
        3.62896171e-05, 5.57005660e-06, 1.08602126e-05, 1.25545421e-05,
        3.79327889e-06, 1.02587012e-06, 5.08045241e-06, 2.45374690e-06,
        4.70524219e-06, 2.72324378e-05, 7.36371294e-05, 1.94362452e-04,
        8.54798054e-05, 3.45071021e-05, 1.22360348e-06, 3.69059239e-06,
        7.40824726e-06, 1.68177721e-05, 5.21816173e-06, 1.43289947e-06,
        1.32765254e-05, 1.09468729e-05, 6.72811893e-06, 1.12544365e-06,
        4.91256287e-06, 7.44649697e-06, 6.48126934e-06, 1.59870399e-06,
        2.07592948e-05, 3.59704950e-06, 4.00522867e-06, 1.055668

In [5]:
CLASS_INDEX = None

In [6]:
CLASS_INDEX_PATH = 'C:/Users/Ashar/Desktop/cdd/imagenet_class_index.json'
#CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

In [7]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [16]:
print (get_predictions(preds, top=5))

[[('n03930630', 'pickup', 0.46586084), ('n03594945', 'jeep', 0.13453923), ('n02814533', 'beach_wagon', 0.10269857), ('n03770679', 'minivan', 0.09930306), ('n02974003', 'car_wheel', 0.038827788)]]


In [ ]:
#First Check - category list creation

In [14]:
from collections import Counter, defaultdict
import os
import pickle as pk

In [10]:
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir('D:/car-damage-dataset/car-damage-dataset/data0/01-whole')
    for i, img_path in enumerate(img_list):
        img = prepare_image('D:/car-damage-dataset/car-damage-dataset/data0/01-whole/'+img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
            print (i, '/', len(img_list), 'complete')
    return Counter(d)

In [11]:
cat_counter=get_car_categories()

0 / 920 complete
100 / 920 complete
200 / 920 complete
300 / 920 complete
400 / 920 complete
500 / 920 complete
600 / 920 complete
700 / 920 complete
800 / 920 complete
900 / 920 complete


In [15]:
#with open('cat_counter.pk', 'wb') as f:
 #   pk.dump(cat_counter,f,-1)

In [42]:
# This piece of code will be used to load already saved cat_counter
#with open('cat_counter.pk', 'rb') as f:
 #   cat_counter = pk.load(f)

In [16]:
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [17]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

# First Check - car or not

In [23]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    print ("Validating that this is a picture of your car...")
    for j in top[0]:
        if j[0:2] in cat_list:
            print (j[0:2])
            return "Validation complete - proceed to damage evaluation"
    return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again."

In [25]:
car_categories_gate('C:/Users/Ashar/Desktop/image2.jpg', cat_list)

Validating that this is a picture of your car...


'Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again.'